In [6]:
# Get all Team IDs
from nba_api.stats.static import teams
import pandas as pd
import time
pd.set_option('display.max_columns', None) 

In [4]:
team_dict = teams.get_teams()
id_team_pairs = [(team['id'], team['full_name']) for team in team_dict]
team_ids = [pair[0] for pair in id_team_pairs]
#team_ids

In [8]:
# Get all team Stats
from nba_api.stats.endpoints import leaguegamefinder

all_teams_data_frames = []

# Loop through each team ID and fetch the corresponding team stats
for team_id in team_ids:
    team_games = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id)
    time.sleep(.600)
    team_data_frame = team_games.get_data_frames()[0]
    all_teams_data_frames.append(team_data_frame)

In [13]:
#all_teams_data_frames[0]

In [14]:
# Get all Player ID
from nba_api.stats.static import players

player_dict = players.get_players()
id_player_pairs = [(player['id'], player['full_name']) for player in player_dict]
player_ids = [pair[0] for pair in id_player_pairs]
#player_ids

In [15]:
from concurrent.futures import ThreadPoolExecutor
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.library.parameters import SeasonAll
from retrying import retry

@retry(stop_max_attempt_number=3, wait_fixed=2000)

def fetch_player_data(player_id):
    player_gamelog = playergamelog.PlayerGameLog(player_id, season=SeasonAll.all)
    time.sleep(.600)
    return player_gamelog.get_data_frames()[0]

with ThreadPoolExecutor() as executor:
    all_players_data_frames = list(executor.map(fetch_player_data, player_ids))

ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

In [16]:
# Loop through each player ID and fetch the corresponding team stats
all_players_data_frames = []
for player_id in player_ids:
    player_gamelog = playergamelog.PlayerGameLog(player_id, season = SeasonAll.all)
    time.sleep(.600)
    player_gamelog_df = player_gamelog.get_data_frames()[0]
    all_players_data_frames.append(player_gamelog_df)
    print(len(all_players_data_frames))

KeyboardInterrupt: 

In [80]:
from concurrent.futures import ThreadPoolExecutor
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.library.parameters import SeasonAll
import concurrent, time

# Function to fetch player game logs for a given player ID
def fetch_player_gamelog(player_id):
    player_gamelog = playergamelog.PlayerGameLog(player_id, season=SeasonAll.all)
    player_gamelog_df = player_gamelog.get_data_frames()[0]
    return player_gamelog_df

# Using ThreadPoolExecutor for concurrent execution
all_players_data_frames = []
with ThreadPoolExecutor() as executor:
    # Fetch player game logs concurrently
    futures = [executor.submit(fetch_player_gamelog, player_id) for player_id in player_ids]

    # Collect the results as they become available
    for future in concurrent.futures.as_completed(futures):
        player_gamelog_df = future.result()
        all_players_data_frames.append(player_gamelog_df)
        print(len(all_players_data_frames))

KeyboardInterrupt: 